<a href="https://colab.research.google.com/github/StevenLevine-NOAA/NBM-Verif/blob/main/Examine_the_Bulls_Eye.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook looks at verious METAR and RAWS sites to help the user determine whether an ob driven measle/bulls eye exists at that point.

The code identifies the grid point in question and the 8 points surround it, interrogating all of them and plotting to derive a potential bulls eye.

Another option/cell shows the magnitude of the bulls eye over time.

Note that for now, you will need to know the i/j location of the ob site in the NDFD/NBM grid for this to work.

In [ ]:
#@title Initialize Notebook Part 1
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@title Initialize Notebook Part 2
!mamba install -q -c conda-forge cartopy contextily pyproj pyepsg pygrib geopy #netCDF4
import numpy as np
#from scipy.interpolate import CubicSpline as cs, UnivariateSpline as us
import pandas as pd
from urllib.request import urlretrieve, urlopen
import requests
from datetime import datetime, timedelta
import pygrib
import pyproj
from pyproj import Proj, transform
import os, re, traceback
from geopy import distance

import matplotlib
from matplotlib.colors import LinearSegmentedColormap
#from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.axes as maxes
import matplotlib.patheffects as PathEffects
import matplotlib.dates as mdates
from matplotlib.path import Path
from matplotlib.textpath import TextToPath
import matplotlib.gridspec as gridspec
from matplotlib.font_manager import FontProperties
matplotlib.rcParams['font.sans-serif'] = 'Liberation Sans'
matplotlib.rcParams['font.family'] = "sans-serif"
from matplotlib.cm import get_cmap
#import seaborn as sns

from cartopy import crs as ccrs, feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
import contextily as cx
import itertools

import warnings
warnings.filterwarnings("ignore")

For Date set up, core_valid_time should be a multiple of 24, beginning with 48 for minT and 36 for maxT.  All forecasts originate at 12Z.

In [7]:
#@title Date Set Up
element = "mint" #@param ["maxt","mint"]
start_date = "2023-02-01" #@param {type:"date"}
nbm_start_date=datetime.strptime(start_date,'%Y-%m-%d')
end_date = "2023-03-01" #@param {type:"date"}
nbm_end_date=datetime.strptime(end_date,'%Y-%m-%d')
core_valid_time = 48 #@param {type:"integer"}
#core time should be muliple of 24, beginning with 48 for mint and 36 for maxt
point_i = 775 #@param {type:"integer"}
point_j = 379 #@param {type:"integer"}
nbm_init_hour = 12 
if element == "maxt":
  nbm_core_valid_hour="00"
  #core_init = nbm_init + timedelta(hours=7)
elif element == "mint":
  nbm_core_valid_hour="12"
  #core_init = nbm_init + timedelta(hours=7)
elif element == "qpf":
  nbm_core_valid_hour="12"

gridBox = {}
gridBox["NorthWest"] = (point_i-1,point_j+1)
gridBox["North"] = (point_i,point_j+1)
gridBox["NorthEast"] = (point_i+1,point_j+1)
gridBox["West"] = (point_i-1,point_j)
gridBox["Point"] = (point_i,point_j)
gridBox["East"] = (point_i+1,point_j)
gridBox["SouthWest"] = (point_i-1,point_j-1)
gridBox["South"] = (point_i,point_j-1)
gridBox["SouthEast"] = (point_i+1,point_j-1)

In [ ]:
#@title Pull the Gribs

###############################
########Subroutines############
###############################

#subroutines - download_subset
def download_subset(remote_url, remote_file, local_filename):
  print("   > Downloading a subset of NBM gribs")
  local_file = "nbm/"+local_filename
  if "qmd" in remote_file:
    if element == "maxt":
      if (int(nbm_qmd_forecasthour_start) % 24 == 0) and (int(nbm_qmd_forecasthour) % 24 ==0):
        search_string = f':TMP:2 m above ground:{str(int(int(nbm_qmd_forecasthour_start)/24))}-{str(int(int(nbm_qmd_forecasthour)/24))} day max fcst:'
      else:
        search_string = f':TMP:2 m above ground:{str(int(nbm_qmd_forecasthour_start))}-{str(int(nbm_qmd_forecasthour))} hour max fcst:'
    elif element == "mint":
      if (int(nbm_qmd_forecasthour_start) % 24 == 0) and (int(nbm_qmd_forecasthour) % 24 ==0):
        search_string = f':TMP:2 m above ground:{str(int(int(nbm_qmd_forecasthour_start)/24))}-{str(int(int(nbm_qmd_forecasthour)/24))} day min fcst:'
      else:
        search_string = f':TMP:2 m above ground:{str(int(nbm_qmd_forecasthour_start))}-{str(int(nbm_qmd_forecasthour))} hour min fcst:'
    elif element == "qpf":
      if (int(nbm_qmd_forecasthour_start) % 24 == 0) and (int(nbm_qmd_forecasthour) % 24 ==0):
        search_string = f':APCP:surface:{str(int(int(nbm_qmd_forecasthour_start)/24))}-{str(int(int(nbm_qmd_forecasthour)/24))} day acc fcst:'
      else:
        search_string = f':APCP:surface:{str(int(nbm_qmd_forecasthour_start))}-{str(int(nbm_qmd_forecasthour))} hour acc fcst:'
  elif "core" in remote_file:
    if element == "maxt":
      search_string = f':TMAX:2 m above ground:{str(int(nbm_core_forecasthour_start))}-{str(int(nbm_core_forecasthour))} hour max fcst:'
    elif element == "mint":
      search_string = f':TMIN:2 m above ground:{str(int(nbm_core_forecasthour_start))}-{str(int(nbm_core_forecasthour))} hour min fcst:'
    elif element == "temp":
      search_string = f':TMP:2 m above ground:{str(int(nbm_core_forecasthour))} hour fcst:'
    elif element == "dwpt":
      search_string = f':DWPT:2 m above ground:{str(int(nbm_core_forecasthour))} hour fcst:'
    elif element == "wspd":
      search_string = f':WIND:10 m above ground:{str(int(nbm_core_forecasthour))} hour fcst:'
  #print(search_string)
  idx = remote_url+".idx"
  r = requests.get(idx)
  if not r.ok: 
    print('     ❌ SORRY! Status Code:', r.status_code, r.reason)
    print(f'      ❌ It does not look like the index file exists: {idx}')

  lines = r.text.split('\n')
  expr = re.compile(search_string)
  byte_ranges = {}
  for n, line in enumerate(lines, start=1):
      # n is the line number (starting from 1) so that when we call for 
      # `lines[n]` it will give us the next line. (Clear as mud??)

      # Use the compiled regular expression to search the line
      if expr.search(line):   
          # aka, if the line contains the string we are looking for...

          # Get the beginning byte in the line we found
          parts = line.split(':')
          rangestart = int(parts[1])

          # Get the beginning byte in the next line...
          if n+1 < len(lines):
              # ...if there is a next line
              parts = lines[n].split(':')
              rangeend = int(parts[1])
          else:
              # ...if there isn't a next line, then go to the end of the file.
              rangeend = ''

          # Store the byte-range string in our dictionary, 
          # and keep the line information too so we can refer back to it.
          byte_ranges[f'{rangestart}-{rangeend}'] = line
          #print(line)
  for i, (byteRange, line) in enumerate(byte_ranges.items()):
        
        if i == 0:
            # If we are working on the first item, overwrite the existing file.
            curl = f'curl -s --range {byteRange} {remote_url} > {local_file}'
        else:
            # If we are working on not the first item, append the existing file.
            curl = f'curl -s --range {byteRange} {remote_url} >> {local_file}'
        try:    
          num, byte, date, var, level, forecast, _ = line.split(':')
        except:
          pass
        
        #print(f'  Downloading GRIB line [{num:>3}]: variable={var}, level={level}, forecast={forecast}')    
        os.system(curl)
    
  if os.path.exists(local_file):
      print(f'      ✅ Success! Searched for [{search_string}] and got [{len(byte_ranges)}] GRIB fields and saved as {local_file}')
      return local_file
  else:
      print(print(f'      ❌ Unsuccessful! Searched for [{search_string}] and did not find anything!'))


#subroutine - daterange
def daterange(start_date,end_date):
  for n in range(int((end_date - start_date).days) + 1):
    yield start_date + timedelta(n)

#subroutine - K to F
def K_to_F(kelvin):
  fahrenheit = 1.8*(kelvin-273)+32.
  return fahrenheit

#subroutine - mm to in
def mm_to_in(millimeters):
  inches = millimeters * 0.0393701
  return inches

#########################################
#######Start of actual processing########
#########################################
fcst=pd.DataFrame(columns=["dates","SouthWest","South","SouthEast","West","Point","East","NorthWest","NorthEast"])
dtdate=[]
lls=[]
lcs=[]
lrs=[]
cls=[]
ccs=[]
crs=[]
uls=[]
ucs=[]
urs=[]
for nbmdate in daterange(nbm_start_date,nbm_end_date):
  dtdate.append(nbmdate)
  nbm_init=nbmdate+timedelta(hours=int(nbm_init_hour))
  nbm_core_forecasthour=core_valid_time
  if element == "mint" or element == "maxt":
    core_init = nbm_init + timedelta(hours=7)
    nbm_core_valid_end_datetime = nbm_init + timedelta(hours=int(nbm_core_forecasthour))
    nbm_core_fhdelta = nbm_core_valid_end_datetime - core_init
  else:
    core_init = nbm_init
    nbm_core_valid_end_datetime = nbm_init
    nbm_core_fhdelta = nbm_core_valid_end_datetime - core_init
  nbm_core_forecasthour = nbm_core_fhdelta.total_seconds() / 3600.
  nbm_core_forecasthour_start = nbm_core_forecasthour - 12
  #get file names
  nbm_init_filen_core = core_init.strftime('%Y%m%d') + "_" + core_init.strftime('%H')
  nbm_url_base = "https://noaa-nbm-grib2-pds.s3.amazonaws.com/blend."+nbm_init.strftime('%Y%m%d') \
              +"/"+nbm_init.strftime('%H')+"/"
  nbm_url_base_core = "https://noaa-nbm-grib2-pds.s3.amazonaws.com/blend."+core_init.strftime('%Y%m%d') \
              +"/"+core_init.strftime('%H')+"/"
  temp_vars = ["maxt","mint","temp","dwpt"]
  #if (element == "qpf"):
  #  detr_file = f'blend.t{int(nbm_init_hour):02}z.qmd.f{int(nbm_qmd_forecasthour):03}.co.grib2'
  #  detr_file_subset = f'blend.t{int(nbm_init_hour):02}z.qmd.{nbm_init_filen}{nbm_init_filen}f{int(nbm_qmd_forecasthour):03}.co.{element}_subset.grib2'
  #  if (int(nbm_qmd_forecasthour) < 12):
  #    break
  #  detr_url = nbm_url_base+"qmd/"+detr_file

  # elif any(te in element for te in temp_vars):
  if any(te in element for te in temp_vars):
    detr_file = f"blend.t{int(core_init.strftime('%H')):02}z.core.f{int(nbm_core_forecasthour):03}.co.grib2"
    detr_file_subset = f"blend.t{int(core_init.strftime('%H')):02}z.core.{nbm_init_filen_core}f{int(nbm_core_forecasthour):03}.co.{element}_subset.grib2"
    if (int(nbm_core_forecasthour) < 12):
      break
    detr_url = nbm_url_base_core+"core/"+detr_file

  #download file if we don't have it yet
  if os.path.exists("nbm/"+detr_file_subset):
    print("   > NBM deterministic already exists")
  else:
    print("   > Getting NBM deterministic")
    if os.path.exists("nbm"):
      pass
    else:
      os.mkdir("nbm")
    download_subset(detr_url, detr_file, detr_file_subset)
  #extract deterministic values
  nbmd = pygrib.open("nbm/"+detr_file_subset)
  if element == "maxt":
    deterministic = nbmd.select(name="Maximum temperature",lengthOfTimeRange=12, stepTypeInternal="max")[0]
    deterministic_array = K_to_F(deterministic.values)
  elif element == "mint":
    deterministic = nbmd.select(name="Minimum temperature",lengthOfTimeRange=12, stepTypeInternal="min")[0]
    deterministic_array = K_to_F(deterministic.values)
  elif element == "qpf":
    deterministic = nbmd.select(name="Total Precipitation",lengthOfTimeRange=24)[-1]
    deterministic_array = mm_to_in(deterministic.values)
  nbmlats, nbmlons = deterministic.latlons()
  #testlat=nbmlats[point_i,point_j]
  #testlon=nbmlons[point_i,point_j]
  #print("Test lat/lon = " + str(testlat) + ", " + str(testlon))
  nbmd.close()
  lls.append(deterministic_array[gridBox["SouthWest"]])
  lcs.append(deterministic_array[gridBox["South"]])
  lrs.append(deterministic_array[gridBox["SouthEast"]])
  cls.append(deterministic_array[gridBox["West"]])
  ccs.append(deterministic_array[gridBox["Point"]])
  crs.append(deterministic_array[gridBox["East"]])
  uls.append(deterministic_array[gridBox["NorthWest"]])
  ucs.append(deterministic_array[gridBox["North"]])
  urs.append(deterministic_array[gridBox["NorthEast"]])
fcst["dates"] = dtdate
fcst["SouthWest"] = lls
fcst["South"] = lcs
fcst["SouthEast"] = lrs
fcst["West"] = cls
fcst["Point"] = ccs
fcst["East"] = crs
fcst["NorthWest"] = uls
fcst["North"] = ucs
fcst["NorthEast"] = urs
fcst.set_index("dates",inplace=True)

In [ ]:
#@title Calculate Differences
fcst["DMax"]=abs(fcst.loc[:,~fcst.columns.isin(['Point'])].max(axis=1)-fcst['Point'])
fcst["DMin"]=abs(fcst.loc[:,~fcst.columns.isin(['Point','DMax'])].min(axis=1)-fcst['Point'])
fcst["Diff"]=fcst[["DMax","DMin"]].min(axis=1)
fcst

In [ ]:
#@title Make Time Series Plot Of Values
fig=plt.figure(constrained_layout=False,figsize=(22,16),dpi=80)
grid=fig.add_gridspec(1,1)
ax=fig.add_subplot(1,1,1)
pts = gridBox.keys()
coldict={"NorthWest":"purple","North":"red","NorthEast":"darkorange","East":"yellow","SouthEast":"yellowgreen","South":"green","SouthWest":"cyan","West":"blue","Point":"black"}
for npoint in coldict.keys():
  fcst.plot(ax=ax,use_index=True,y=npoint,label=npoint,grid=True,marker='o',markersize=3,linewidth=2,color=coldict[npoint])
ax.set_xlabel("Forecast Made",fontdict={'fontsize':20})
ax.set_ylabel("Forecast Value",fontdict={'fontsize':20})
#ax.xaxis.set_major_locator(mdates.DayLocator())
#ax.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
#ax.tick_params(axis='both',which='both',labelsize=15)
#ax.grid(visible='True',axis='both')
#ax2=ax.twinx()
#fcst.plot(ax=ax,use_index=True,y="Diff",marker='o',markersize=3,linewidth=2,color='black',linestyle='dashed')
#ax2.set_ylabel("Point/Surrounding Min",fontdict={'fontsize':20})
ax.tick_params(axis='y',which='both',labelsize=15)
ax.xaxis.set_major_locator(mdates.DayLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
ax.tick_params(axis='both',which='both',labelsize=15)
ax.grid(visible='True',axis='both')
ax.legend(loc='lower center',bbox_to_anchor=[0.5,1.00],fancybox=True,shadow=True,fontsize='x-large',ncol=9)
#ax.axvline(datetime(2023,2,3),linestyle='dashed',color='magenta')
title_string="Time series of " + str(core_valid_time) + " hour " + element + " forecasts valid near site"
fig.text(0.5,0.92,title_string,horizontalalignment='center',verticalalignment='bottom',weight='bold',fontsize=20)
fig.savefig('neighbor_points.png')

In [ ]:
#@title Make Time Series of Difference
figb=plt.figure(constrained_layout=False,figsize=(22,16))
grid=figb.add_gridspec(1,1)
axb=figb.add_subplot(1,1,1)
fcst.plot(ax=axb,use_index=True,y="Diff",marker='o',markersize=3,linewidth=2,color='black',linestyle='dashed')
axb.set_xlabel("Forecast Made",fontdict={'fontsize':20})
axb.set_ylabel("Point/Surrounding Difference",fontdict={'fontsize':20})
axb.tick_params(axis='both',which='both',labelsize=15)
axb.xaxis.set_major_locator(mdates.DayLocator())
axb.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))
axb.grid(visible='True',axis='both')
#axb.axvline(datetime(2023,2,3),linestyle='dashed',color='magenta')
title_string="Time Series of minimum difference between point and surroundings"
figb.text(0.5,0.92,title_string,horizontalalignment='center',verticalalignment='bottom',weight='bold',fontsize=20)
figb.savefig('bullseye_diff.png')